<a href="https://colab.research.google.com/github/HiagoAD/GDI2/blob/master/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

## Load datasets


In [158]:
dengue = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2020.csv", encoding ="UTF-8")
emlurb = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2020.csv", encoding ="UTF-8")

dengue19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2019.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2019.csv", encoding ="UTF-8")

dengue18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2018.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2018.csv", encoding ="UTF-8")

dengue17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2017.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2017.csv", encoding ="UTF-8")

dengue16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2016.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2016.csv", encoding ="UTF-8")


b'Skipping line 108: expected 126 fields, saw 127\nSkipping line 1485: expected 126 fields, saw 127\nSkipping line 2584: expected 126 fields, saw 127\nSkipping line 4415: expected 126 fields, saw 127\nSkipping line 5659: expected 126 fields, saw 127\nSkipping line 6469: expected 126 fields, saw 127\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (55,113) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Dados de 2020 - 2016


Iremos fazer os mesmos tratamentos anteriores para os dados de 2019 até 2016.

In [197]:
# Dados Dengue

merge_dengue = dengue.append([dengue19,dengue18,dengue17,dengue16], ignore_index = True)
merge_dengue = merge_dengue[["no_bairro_residencia", "febre","dt_notificacao"]]
merge_dengue['dt_notificacao'] = pd.to_datetime(merge_dengue['dt_notificacao'])

# Agrupamos por número de casos por dia

merge_dengue = merge_dengue.groupby([pd.Grouper(key="dt_notificacao", freq="D")]).size().reset_index(name="CASOS_DENGUE")


# Dados Emlurb

merge_emlurb = emlurb_data.append([emlurb19,emlurb18,emlurb17,emlurb16], ignore_index = True)
merge_emlurb = merge_emlurb[["GRUPOSERVICO_DESCRICAO", "BAIRRO", "DATA_DEMANDA"]]
merge_emlurb = merge_emlurb[(merge_emlurb.GRUPOSERVICO_DESCRICAO == "DRENAGEM")]
merge_emlurb["DATA_DEMANDA"] = pd.to_datetime(merge_emlurb["DATA_DEMANDA"],errors='coerce')
merge_emlurb.dropna(inplace=True)

# Agrupamos por número de solicitações por dia

#merge_emlurb = merge_emlurb.groupby([pd.Grouper(key="DATA_DEMANDA", freq="D")]).size().reset_index(name="CHAMAD0S_EMLURB")

#year_filter = merge_emlurb[(merge_emlurb['DATA_DEMANDA'].dt.year == 2018)]

#merge_emlurb
merge_dengue

,dt_notificacao,CASOS_DENGUE
0,2016-01-03,8
1,2016-01-04,39
2,2016-01-05,35
3,2016-01-06,51
4,2016-01-07,79
...,...,...
1656,2020-07-16,4
1657,2020-07-17,1
1658,2020-07-18,1
1659,2020-07-19,1


## Feature de predição

1.   Número de casos de dengue por dia.
2.   Número de solicitações de drenagem por dia.



## Separação de dados: Treino, Validação e Teste

Texto.

In [193]:
def get_values(df, column_name):
  X = df.drop(columns=[column_name]).values
  y = df[column_name].values

  return X, y

In [198]:
train, validation, test = np.split(merge_dengue.sample(frac=1, random_state=42), [int(.6*len(merge_dengue)), int(.8*len(merge_dengue))])

column_name = 'CASOS_DENGUE'

X_train, y_train = get_values(train, column_name)
X_valid, y_valid = get_values(validation, column_name)
X_test,  y_test  = get_values(test, column_name)

In [200]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(996, 1)
(333, 1)
(332, 1)


## Escolha de Algoritmos


*   Random Forest
*   MLP
*   KNN



## MLFlow

Texto.

## Seleção de hiper-parâmetros

Texto.


## Diagnóstico de modelos

Texto.


# **MLP**

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers, optimizers, metrics, activations

resolver = tensorflow.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tensorflow.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tensorflow.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tensorflow.config.list_logical_devices('TPU'))

In [ ]:
def MLP():
    nu_shape = input_shape[0] * input_shape[1]
    curve = activations.sigmoid
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape),
            layers.Reshape((nu_shape,)),
            layers.Dense(nu_shape, activation=curve),
            layers.Dense(nu_shape, activation=curve),
            layers.Dropout(0.1),
            layers.Dense(nu_shape, activation=curve),
            layers.Dense(num_classes, activation=activations.softmax)
        ]
    )
    model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(learning_rate=1), metrics=["accuracy", metrics.Recall(), metrics.Precision()], steps_per_execution=50)

    return model

In [ ]:
def LSTM(hidden_layer_size, look_back = 7):
  model = Sequential()
  model.add(LSTM(hidden_layer_size, input_shape=(look_back,1)))
  model.add(Dense(1)) # to have a regressor
  model.compile(loss='mean_squared_error', optimizer='adam', steps_per_execution=50)
  return model